<a href="https://colab.research.google.com/github/liud22/datascienceIBM/blob/main/Capstone_P2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

####We will start with loading data

There are two ways for crawling here

In [59]:
#direct import
import pandas as pd
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [60]:
import requests
from bs4 import BeautifulSoup
import numpy as np

In [61]:
# define data list and headline list
wikidata = []
columns = []

In [62]:
# define link to be crawled
page_crawl = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(page_crawl.text, 'lxml')
table = soup.find(class_='wikitable')

# collect each th and td and organize in to a table
for index, item in enumerate(table.find_all('tr')):
    row = []
    if (index == 0):
      columns = row
    for td in item.find_all(['th','td']):
      content = td.text
      #print(content)
      content_st = content.rstrip()
      row.append(content_st)
    #print(row)
    if (index != 0):
        wikidata.append(row)
        

df = pd.DataFrame(data = wikidata,columns = columns)
df = df.drop_duplicates() 
df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


Clean the data

In [63]:
#Clean and prepare the data
#filter out the "non assigned" in "Borough"
df = df[df['Borough'] != 'Not assigned']
"combine Neighbourhood with the same postal code"
df["Neighbourhood"] = df.groupby("Postal Code")["Neighbourhood"].transform(lambda x: ', '.join(x))
df = df.drop_duplicates()  
df 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [64]:
df.replace("Not assigned",df.Borough,inplace=True)
df

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [65]:
co_df = pd.read_csv("http://cocl.us/Geospatial_data")
co_df

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


*italicized text*

In [66]:
#result = pd.concat([df, co_df], join='left')

result = pd.merge(df, co_df, on='Postal Code')
result

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [67]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library

In [68]:
!pip install geocoder

In [69]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Folium installed and imported!')
print('GeoJSON file downloaded!')

/bin/bash: conda: command not found
Folium installed and imported!
GeoJSON file downloaded!


In [70]:
# for the city Toronto, latitude and longtitude are manually extracted via google search
toronto_latitude = 43.6532; 
toronto_longitude = -79.3832
toronto_folim = folium.Map(location = [toronto_latitude, toronto_longitude], zoom_start = 10)

# map the Borough per neighbourhood

In [81]:

# add markers to map
for lat, lng, bor, neigh in zip(result['Latitude'], result['Longitude'], result['Borough'], result['Neighbourhood']):
    label = 'Neighbourhood:{}, Borough:{}'.format(neigh, bor)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=len(neigh.split(","))*2,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(toronto_folim)  
    



In [72]:
toronto_folim
#display the map here

In [73]:
#user detailes
CLIENT_ID = 'zzz' # your Foursquare ID
CLIENT_SECRET = 'zzz' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YS0EHDWGNFJPJOPNOCQFVYJTRDNIH3HWPWRSTPZTEMQMOSXA
CLIENT_SECRET:FHT3FSKTZUJ0COSRB0FQ3YVOKOCPEA52ULOGY0SHJIPPYLHG


## We are intested in how the similar the neighbors are based on venues features. We wii take Etobicoke as an example. 

In [74]:
# We will explore and cluster the venues in Etobicoke.
Etobicoke= result[result['Borough'].str.contains('Etobicoke', na = False)].reset_index(drop=True)
Etobicoke

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
1,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",43.650943,-79.554724
2,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201
3,M9P,Etobicoke,Westmount,43.696319,-79.532242
4,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
5,M8V,Etobicoke,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321
6,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
7,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484
8,M9W,Etobicoke,"Northwest, West Humber - Clairville",43.706748,-79.594054
9,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944


we will create a function to repeatly deal with the neighborhoods in Etobicoke to exlore the venues

In [75]:
Etobicoke.shape
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius_defined = 400
def getNearbyVenues(names, latitudes, longitudes, radius=radius_defined):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [76]:
Etobicoke_venues = getNearbyVenues(names=Etobicoke['Neighbourhood'],
                                   latitudes=Etobicoke['Latitude'],
                                   longitudes=Etobicoke['Longitude']
                                  )

Islington Avenue, Humber Valley Village
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Westmount
Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens
New Toronto, Mimico South, Humber Bay Shores
South Steeles, Silverstone, Humbergate, Jamestown, Mount Olive, Beaumond Heights, Thistletown, Albion Gardens
Alderwood, Long Branch
Northwest, West Humber - Clairville
The Kingsway, Montgomery Road, Old Mill North
Old Mill South, King's Mill Park, Sunnylea, Humber Bay, Mimico NE, The Queensway East, Royal York South East, Kingsway Park South East
Mimico NW, The Queensway West, South of Bloor, Kingsway Park South West, Royal York South West


In [77]:
Etobicoke_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"West Deane Park, Princess Gardens, Martin Grov...",43.650943,-79.554724,Majestic Tees,43.649774,-79.555437,Print Shop
1,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201,LCBO,43.642099,-79.576592,Liquor Store
2,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201,Starbucks,43.641312,-79.576924,Coffee Shop
3,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201,The Beer Store,43.641313,-79.576925,Beer Store
4,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201,Shoppers Drug Mart,43.641312,-79.576924,Pharmacy
5,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201,Pizza Hut,43.641845,-79.576556,Pizza Place
6,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201,Cafe Sympatico,43.641820,-79.576721,Café
7,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201,Burnhamthorpe Mall,43.641741,-79.576891,Shopping Plaza
8,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201,Hasty Market,43.641871,-79.576370,Convenience Store
9,Westmount,43.696319,-79.532242,Starbucks,43.696338,-79.533398,Coffee Shop


In [78]:

print('There are {} uniques categories.'.format(len(Etobicoke_venues['Venue Category'].unique())))

There are 32 uniques categories.


In [79]:
Etobicoke_venues.groupby('Venue Category').count()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Bakery,1,1,1,1,1,1
Bar,1,1,1,1,1,1
Baseball Field,1,1,1,1,1,1
Beer Store,1,1,1,1,1,1
Burger Joint,1,1,1,1,1,1
Bus Line,1,1,1,1,1,1
Café,1,1,1,1,1,1
Chinese Restaurant,1,1,1,1,1,1
Coffee Shop,4,4,4,4,4,4


In [80]:
# one hot encoding
Etobicoke_onehot = pd.get_dummies(Etobicoke_venues[['Venue Category']], prefix="", prefix_sep="")
Etobicoke_onehot['Neighborhood'] = Etobicoke_venues['Neighborhood'] 

# define neighborhood as the index
cols=list(Etobicoke_onehot.columns.values)
cols.pop(cols.index('Neighborhood'))
Etobicoke_onehot=Etobicoke_onehot[['Neighborhood']+cols]

#Aggregate the data
Etobicoke_grouped = Etobicoke_onehot.groupby('Neighborhood').sum().reset_index()
Etobicoke_grouped

,Neighborhood,Bakery,Bar,Baseball Field,Beer Store,Burger Joint,Bus Line,Café,Chinese Restaurant,Coffee Shop,Convenience Store,Discount Store,Fast Food Restaurant,Fried Chicken Joint,Grocery Store,Gym,Hardware Store,Kids Store,Liquor Store,Park,Pharmacy,Pizza Place,Pool,Print Shop,Pub,Rental Car Location,Sandwich Place,Shopping Plaza,Social Club,Supplement Shop,Tanning Salon,Thrift / Vintage Store,Wings Joint
0,"Alderwood, Long Branch",0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0,0,1,0,1,0,0,0,0,0,0
1,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",0,0,0,1,0,0,1,0,1,1,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1,0,0,0,0,0
2,"Kingsview Village, St. Phillips, Martin Grove ...",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Mimico NW, The Queensway West, South of Bloor,...",1,0,0,0,1,0,0,0,0,1,1,1,0,1,0,1,1,0,0,0,0,0,0,0,0,1,0,1,1,1,1,1
4,"New Toronto, Mimico South, Humber Bay Shores",0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,"Northwest, West Humber - Clairville",0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
6,"Old Mill South, King's Mill Park, Sunnylea, Hu...",0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,"South Steeles, Silverstone, Humbergate, Jamest...",0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,2,0,0,0,0,1,0,0,0,0,0,0
8,"The Kingsway, Montgomery Road, Old Mill North",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
9,"West Deane Park, Princess Gardens, Martin Grov...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


#We will look at how these neightbors are similar in their venues setting

In [46]:
Etobicoke_grouped.set_index("Neighborhood")

,Bakery,Bar,Baseball Field,Beer Store,Burger Joint,Bus Line,Café,Chinese Restaurant,Coffee Shop,Convenience Store,Discount Store,Fast Food Restaurant,Fried Chicken Joint,Grocery Store,Gym,Hardware Store,Kids Store,Liquor Store,Park,Pharmacy,Pizza Place,Pool,Print Shop,Pub,Rental Car Location,Sandwich Place,Shopping Plaza,Social Club,Supplement Shop,Tanning Salon,Thrift / Vintage Store,Wings Joint
Neighborhood,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Alderwood, Long Branch",0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0,0,1,0,1,0,0,0,0,0,0
"Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood",0,0,0,1,0,0,1,0,1,1,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1,0,0,0,0,0
"Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
"Mimico NW, The Queensway West, South of Bloor, Kingsway Park South West, Royal York South West",1,0,0,0,1,0,0,0,0,1,1,1,0,1,0,1,1,0,0,0,0,0,0,0,0,1,0,1,1,1,1,1
"New Toronto, Mimico South, Humber Bay Shores",0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"Northwest, West Humber - Clairville",0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
"Old Mill South, King's Mill Park, Sunnylea, Humber Bay, Mimico NE, The Queensway East, Royal York South East, Kingsway Park South East",0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"South Steeles, Silverstone, Humbergate, Jamestown, Mount Olive, Beaumond Heights, Thistletown, Albion Gardens",0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,2,0,0,0,0,1,0,0,0,0,0,0
"The Kingsway, Montgomery Road, Old Mill North",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [56]:
from sklearn.metrics.pairwise import cosine_similarity
simi_df = Etobicoke_grouped[['Bakery', 'Bar', 'Baseball Field', 'Beer Store',
       'Burger Joint', 'Bus Line', 'Café', 'Chinese Restaurant', 'Coffee Shop',
       'Convenience Store', 'Discount Store', 'Fast Food Restaurant',
       'Fried Chicken Joint', 'Grocery Store', 'Gym', 'Hardware Store',
       'Kids Store', 'Liquor Store', 'Park', 'Pharmacy', 'Pizza Place', 'Pool',
       'Print Shop', 'Pub', 'Rental Car Location', 'Sandwich Place',
       'Shopping Plaza', 'Social Club', 'Supplement Shop', 'Tanning Salon',
       'Thrift / Vintage Store', 'Wings Joint']]
simi_result = cosine_similarity(simi_df)
df = pd.DataFrame(data=simi_result)
print(df)


          0         1    2         3   ...        7    8    9         10
0   1.000000  0.433013  0.0  0.109109  ...  0.433013  0.0  0.0  0.612372
1   0.433013  1.000000  0.0  0.094491  ...  0.375000  0.0  0.0  0.353553
2   0.000000  0.000000  1.0  0.000000  ...  0.000000  0.0  0.0  0.000000
3   0.109109  0.094491  0.0  1.000000  ...  0.188982  0.0  0.0  0.133631
4   0.408248  0.353553  0.0  0.000000  ...  0.000000  0.0  0.0  0.500000
5   0.000000  0.000000  0.0  0.000000  ...  0.000000  0.0  0.0  0.000000
6   0.000000  0.000000  0.0  0.000000  ...  0.000000  0.0  0.0  0.000000
7   0.433013  0.375000  0.0  0.188982  ...  1.000000  0.0  0.0  0.530330
8   0.000000  0.000000  0.0  0.000000  ...  0.000000  1.0  0.0  0.000000
9   0.000000  0.000000  0.0  0.000000  ...  0.000000  0.0  1.0  0.000000
10  0.612372  0.353553  0.0  0.133631  ...  0.530330  0.0  0.0  1.000000

[11 rows x 11 columns]


# From this similarity matrix, we can figure out that Neighborhood Alderwood, Long Branch and Neighborhood Westmount has venues similarity over than 0.6